In [1]:
import requests
from textblob import TextBlob

In [2]:
good_news_url = ('http://newsapi.org/v2/everything?'
                 'q=Good News&'
                 'from=2020-07-18&'
                 'pageSize=100&'
                 'sortBy=popularity&'
                 'apiKey=7cbc46d369ec49efba7d4e1bd3df8b1a')

good_news_response = requests.get(good_news_url)

In [3]:
def print_seperate_scores(response, num):
    i = 0
    max_score = 0
    str_save = ""
    while i < len(response.json()['articles']):
        
        article_dict = response.json()['articles'][i]
        text_to_be_analysed = article_dict['title']
        blob = TextBlob(text_to_be_analysed)
        score_title = blob.sentiment.polarity
        if (article_dict['description'] != None):
            text_to_be_analysed = text_to_be_analysed+ ". " + article_dict['description']
            blob = TextBlob(article_dict['description'])
            score_desc = blob.sentiment.polarity
        if (article_dict['content'] != None):
            text_to_be_analysed = text_to_be_analysed+ ". " + article_dict['content']
            blob = TextBlob(article_dict['content'])
            score_content = blob.sentiment.polarity
        blob = TextBlob(text_to_be_analysed)
        score_all = blob.sentiment.polarity


        if score_all >= num or score_title >= num or score_desc >= num or score_content >= num:
            #print(article_dict)
            print("")
            
            
            print("Score (All): " + str(score_all))
            print("")
            print("Text (title): "+ article_dict['title'])
            print("Score (title): " + str(score_title))
            if (article_dict['description'] != None):
                print("")
                print("Text (desc): "+ article_dict['description'])
                print("Score (desc): " + str(score_desc))
            if (article_dict['content'] != None):
                print("")
                print("Text (content): "+ article_dict['content'])
                print("Score (content): " + str(score_content))
            print("")
        i += 1

In [4]:
def add_scores_to_array(array):
    i = 0
    return_array = array[:]
    while (i < len(return_array)):
        article_dict = return_array[i]
        text_to_be_analysed = article_dict['title']

        blob = TextBlob(article_dict['title'])
        score_title = blob.sentiment.polarity
        article_dict['titleScore'] = score_title

        if (article_dict['description'] != None):
            blob = TextBlob(article_dict['description'])
            text_to_be_analysed = text_to_be_analysed+ ". " + article_dict['description']
            score_desc = blob.sentiment.polarity
            article_dict['descriptionScore'] = score_desc
        else:
            score_desc = 0
            article_dict['descriptionScore'] = 0

        if (article_dict['content'] != None):
            text_to_be_analysed = text_to_be_analysed+ ". " + article_dict['content']
            blob = TextBlob(article_dict['content'])
            score_content = blob.sentiment.polarity
            article_dict['contentScore'] = score_content
        else:
            score_content = 0
            article_dict['contentScore'] = 0
        blob = TextBlob(text_to_be_analysed)
        score_all = blob.sentiment.polarity
        article_dict['allScore'] = score_all 
        i += 1
    return return_array

In [5]:
import pandas as pd

In [6]:
array_good_news = add_scores_to_array(good_news_response.json()['articles'])
df = pd.DataFrame(array_good_news)
df1 = df[(df['descriptionScore'] >= 0.5) | (df['titleScore'] >= 0.5) | (df['contentScore'] >= 0.5) | (df['allScore'] >= 0.5)]

df2 = df1[((df1['descriptionScore'] >= 0.01) | (df1['description'].isnull()) | (df1['description']=='')) & ((df1['titleScore'] >= 0.01)| (df1['title'].isnull()) | (df1['title']=='')) & ((df1['contentScore'] >= 0.01)| (df1['content'].isnull()) | (df1['content']=='')) & (df1['allScore'] >= 0.2)]
df2

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
2,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Eric Eldon,Startups Weekly: The TechCrunch List reveals i...,Editor’s note: Get this free weekly recap of T...,http://techcrunch.com/2020/07/18/startups-week...,https://techcrunch.com/wp-content/uploads/2020...,2020-07-18T18:00:22Z,Editors note: Get this free weekly recap of Te...,0.500000,0.359091,0.450000,0.405195
48,"{'id': None, 'name': 'Sky Sports'}",Sky Sports,PL Review Podcast: Best Man Utd team since Fer...,Is the current Manchester United team the best...,https://www.skysports.com/football/news/11095/...,https://e1.365dm.com/20/07/1600x900/skysports-...,2020-07-18T15:00:00Z,Is the current Manchester United team the best...,1.000000,0.500000,0.500000,0.571429
56,"{'id': None, 'name': 'Motley Fool'}",newsfeedback@fool.com (Christy Bieber),3 Key Signs More Stimulus Money Is on the Way ...,There's good news for those hoping for more st...,https://www.fool.com/investing/2020/07/18/3-ke...,https://g.foolcdn.com/editorial/images/581129/...,2020-07-18T10:33:00Z,The IRS has sent out more than $269 billion in...,0.066667,0.662500,0.500000,0.360714
58,"{'id': None, 'name': 'Motley Fool'}",newsfeedback@fool.com (David Jagielski),Better Buy: Rite Aid vs. Walgreens,Given the economic and market conditions cause...,https://www.fool.com/investing/2020/07/18/bett...,https://g.foolcdn.com/editorial/images/582641/...,2020-07-18T11:55:00Z,Investing in pharmacy retailers is a relativel...,0.500000,0.200000,0.281481,0.308889
79,"{'id': None, 'name': 'The Indian Express'}",Lifestyle Desk,"Happy Sawan Shivratri 2020: Wishes, images, st...","Happy Sawan Shivratri 2020 Wishes Images, Quot...",https://indianexpress.com/article/lifestyle/li...,https://images.indianexpress.com/2020/07/Sawan...,2020-07-18T14:30:33Z,Happy Sawan Shivratri 2020 Wishes Images: Happ...,0.800000,0.650000,0.800000,0.740000


In [7]:
df_good_news = df2.copy()
df_good_news

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
2,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Eric Eldon,Startups Weekly: The TechCrunch List reveals i...,Editor’s note: Get this free weekly recap of T...,http://techcrunch.com/2020/07/18/startups-week...,https://techcrunch.com/wp-content/uploads/2020...,2020-07-18T18:00:22Z,Editors note: Get this free weekly recap of Te...,0.500000,0.359091,0.450000,0.405195
48,"{'id': None, 'name': 'Sky Sports'}",Sky Sports,PL Review Podcast: Best Man Utd team since Fer...,Is the current Manchester United team the best...,https://www.skysports.com/football/news/11095/...,https://e1.365dm.com/20/07/1600x900/skysports-...,2020-07-18T15:00:00Z,Is the current Manchester United team the best...,1.000000,0.500000,0.500000,0.571429
56,"{'id': None, 'name': 'Motley Fool'}",newsfeedback@fool.com (Christy Bieber),3 Key Signs More Stimulus Money Is on the Way ...,There's good news for those hoping for more st...,https://www.fool.com/investing/2020/07/18/3-ke...,https://g.foolcdn.com/editorial/images/581129/...,2020-07-18T10:33:00Z,The IRS has sent out more than $269 billion in...,0.066667,0.662500,0.500000,0.360714
58,"{'id': None, 'name': 'Motley Fool'}",newsfeedback@fool.com (David Jagielski),Better Buy: Rite Aid vs. Walgreens,Given the economic and market conditions cause...,https://www.fool.com/investing/2020/07/18/bett...,https://g.foolcdn.com/editorial/images/582641/...,2020-07-18T11:55:00Z,Investing in pharmacy retailers is a relativel...,0.500000,0.200000,0.281481,0.308889
79,"{'id': None, 'name': 'The Indian Express'}",Lifestyle Desk,"Happy Sawan Shivratri 2020: Wishes, images, st...","Happy Sawan Shivratri 2020 Wishes Images, Quot...",https://indianexpress.com/article/lifestyle/li...,https://images.indianexpress.com/2020/07/Sawan...,2020-07-18T14:30:33Z,Happy Sawan Shivratri 2020 Wishes Images: Happ...,0.800000,0.650000,0.800000,0.740000


In [8]:
top_good_url = ('http://newsapi.org/v2/top-headlines?'
                 'q=Good News&'
                 'from=2020-07-18&'
                 'pageSize=100&'
                 'sortBy=popularity&'
                 'apiKey=7cbc46d369ec49efba7d4e1bd3df8b1a')

top_good_response = requests.get(top_good_url)

In [9]:
# Reason for abandonment: not enough result to decide how positive it is

In [10]:
top_good_response.json()

{'status': 'ok',
 'totalResults': 1,
 'articles': [{'source': {'id': 'fortune', 'name': 'Fortune'},
   'author': 'Jennifer Hoewe, Brett Sherrick',
   'title': 'Are you willing to pay for news? The future of journalism may depend on it',
   'description': "Commentary: American journalism in crisis. The public needs to pay for good reporting if there's any hope to save the news media.",
   'url': 'https://fortune.com/2020/07/16/american-journalism-crisis-save-news-media/',
   'urlToImage': 'https://content.fortune.com/wp-content/uploads/2020/07/Future-of-Journalism-NYTimes-Subsciprtion-GettyImages-1202342874.jpg?resize=1200,600',
   'publishedAt': '2020-07-16T18:02:30Z',
   'content': None}]}

In [11]:
good_url = ('http://newsapi.org/v2/top-headlines?'
                 'q=Good&'
                 'from=2020-07-18&'
                 'pageSize=100&'
                 'sortBy=popularity&'
                 'apiKey=7cbc46d369ec49efba7d4e1bd3df8b1a')
good_response = requests.get(good_url)

In [12]:
good_response.json()

{'status': 'ok',
 'totalResults': 10,
 'articles': [{'source': {'id': 'usa-today', 'name': 'USA Today'},
   'author': None,
   'title': "Appreciation: For John Lewis, a lifetime of making 'good trouble' left scars and a legacy",
   'description': 'The iconic civil rights leader was all about bridges, from marching in Selma in 1965 to providing links to a new generation of racial protests today.',
   'url': 'https://www.usatoday.com/story/news/politics/2020/07/18/john-lewis-and-good-trouble-left-scars-and-legacy-nation/5464195002/',
   'urlToImage': 'https://www.gannett-cdn.com/presto/2020/07/14/USAT/91da859e-52d8-4c51-ac40-dd3f2ef1e2fb-XXX_GAN-CR-NASHVILLE-LEWIS-012813-2.jpg?crop=1436,808,x0,y507&width=1600&height=800&fit=bounds',
   'publishedAt': '2020-07-18T19:45:21+00:00',
   'content': 'Rep. John Lewis, as a Nashville college student, was part of a group that played an instrumental role in desegregating the city.\r\nNashville Tennessean\r\nJohn Lewis rubbed the scar on his forehea

In [13]:
#lesson learned: don't use query in top headlines

In [14]:
us_url = ('http://newsapi.org/v2/top-headlines?'
                 'country=us&'
                 'from=2020-07-18&'
                 'pageSize=100&'
                 'sortBy=popularity&'
                 'apiKey=7cbc46d369ec49efba7d4e1bd3df8b1a')
us_response = requests.get(us_url)

In [15]:
array_top_us = add_scores_to_array(us_response.json()['articles'])
df = pd.DataFrame(array_top_us)
df1 = df[(df['descriptionScore'] >= 0.5) | (df['titleScore'] >= 0.5) | (df['contentScore'] >= 0.5) | (df['allScore'] >= 0.5)]
df1

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
14,"{'id': None, 'name': 'New York Daily News'}","Rocco Parascandola, Thomas Tracy",NYPD top cop calls city leaders ‘cowards’ in p...,In a stunning seven-minute video taken during ...,https://www.nydailynews.com/new-york/nyc-crime...,https://www.nydailynews.com/resizer/ORJEmTlTf1...,2020-07-18T16:37:00Z,None,0.212121,0.500000,0.000000,0.284091
24,"{'id': None, 'name': 'MacRumors'}",MacRumors Staff,Top Stories: Apple Warns About MacBook Webcam ...,While much of our attention has been focused o...,https://www.macrumors.com/2020/07/18/top-stori...,https://images.macrumors.com/article-new/2020/...,2020-07-18T13:00:00Z,While much of our attention has been focused o...,0.500000,-0.016667,-0.016667,0.057143
33,"{'id': 'cnn', 'name': 'CNN'}","Tami Luhby, CNN",Appeals court upholds Trump administration's e...,The US District Court of Appeals for the Distr...,https://www.cnn.com/2020/07/17/politics/short-...,https://cdn.cnn.com/cnnnext/dam/assets/2007171...,2020-07-17T22:03:00Z,(CNN)The US District Court of Appeals for the ...,0.000000,0.095833,0.525000,0.238889
37,"{'id': None, 'name': 'Hot Air'}",None,"Walmart, Kroger weigh in on national debate: H...",Setting their own rules.,https://hotair.com/archives/karen-townsend/202...,https://hotair.com/wp/wp-content/uploads/2017/...,2020-07-17T02:01:57Z,Two of the countrys largest retailers have jum...,0.083333,0.600000,0.250000,0.220000


In [16]:
df2 = df1[((df1['descriptionScore'] >= 0.01) | (df1['description'].isnull()) | (df1['description']=='')) & ((df1['titleScore'] >= 0.01)| (df1['title'].isnull()) | (df1['title']=='')) & ((df1['contentScore'] >= 0.01)| (df1['content'].isnull()) | (df1['content']=='')) & (df1['allScore'] >= 0.2)]

df2

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
14,"{'id': None, 'name': 'New York Daily News'}","Rocco Parascandola, Thomas Tracy",NYPD top cop calls city leaders ‘cowards’ in p...,In a stunning seven-minute video taken during ...,https://www.nydailynews.com/new-york/nyc-crime...,https://www.nydailynews.com/resizer/ORJEmTlTf1...,2020-07-18T16:37:00Z,None,0.212121,0.5,0.00,0.284091
37,"{'id': None, 'name': 'Hot Air'}",None,"Walmart, Kroger weigh in on national debate: H...",Setting their own rules.,https://hotair.com/archives/karen-townsend/202...,https://hotair.com/wp/wp-content/uploads/2017/...,2020-07-17T02:01:57Z,Two of the countrys largest retailers have jum...,0.083333,0.6,0.25,0.220000


In [17]:
df_top_us = df2.copy()
df_top_us

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
14,"{'id': None, 'name': 'New York Daily News'}","Rocco Parascandola, Thomas Tracy",NYPD top cop calls city leaders ‘cowards’ in p...,In a stunning seven-minute video taken during ...,https://www.nydailynews.com/new-york/nyc-crime...,https://www.nydailynews.com/resizer/ORJEmTlTf1...,2020-07-18T16:37:00Z,None,0.212121,0.5,0.00,0.284091
37,"{'id': None, 'name': 'Hot Air'}",None,"Walmart, Kroger weigh in on national debate: H...",Setting their own rules.,https://hotair.com/archives/karen-townsend/202...,https://hotair.com/wp/wp-content/uploads/2017/...,2020-07-17T02:01:57Z,Two of the countrys largest retailers have jum...,0.083333,0.6,0.25,0.220000


In [18]:
must_good_url = ('http://newsapi.org/v2/everything?'
                 'q=+good&'
                 'from=2020-07-18&'
                 'pageSize=100&'
                 'sortBy=popularity&'
                 'apiKey=7cbc46d369ec49efba7d4e1bd3df8b1a')

must_good_response = requests.get(must_good_url)

In [19]:
array_must_good = add_scores_to_array(must_good_response.json()['articles'])
df = pd.DataFrame(array_must_good)
df1 = df[(df['descriptionScore'] >= 0.5) | (df['titleScore'] >= 0.5) | (df['contentScore'] >= 0.5) | (df['allScore'] >= 0.5)]
df1

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
5,"{'id': 'wired', 'name': 'Wired'}",Brian Barrett,Who Pulled Off the Twitter Hack?,"Plus: WhatsApp's court case, a VPN exposed, an...",https://www.wired.com/story/twitter-hack-suspe...,https://media.wired.com/photos/5f121d4e09857a7...,2020-07-18T13:00:00Z,"On Wednesday, anunprecedented Twitter hack saw...",0.00000,0.500000,0.500000,0.500000
6,"{'id': 'new-york-magazine', 'name': 'New York ...",Jordan Bowman,"The best electric scooters, according to experts","It’s like regular scooting, only easier.",https://nymag.com/strategist/article/best-elec...,https://pyxis.nymag.com/v1/imgs/452/97c/f2c3dc...,2020-07-18T14:00:00Z,At a time when public transportation has never...,1.00000,0.000000,0.111111,0.222222
7,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Eric Eldon,Startups Weekly: The TechCrunch List reveals i...,Editor’s note: Get this free weekly recap of T...,http://techcrunch.com/2020/07/18/startups-week...,https://techcrunch.com/wp-content/uploads/2020...,2020-07-18T18:00:22Z,Editors note: Get this free weekly recap of Te...,0.50000,0.359091,0.450000,0.405195
8,"{'id': None, 'name': 'Gizmodo.com'}",Alyse Stanley,Researchers Created an App That Monitors Your ...,Even though I appreciate that it’s for a good ...,https://gizmodo.com/researchers-created-an-app...,https://i.kinja-img.com/gawker-media/image/upl...,2020-07-18T04:25:00Z,Even though I appreciate that its for a good c...,-0.10000,0.600000,0.400000,0.380000
11,"{'id': 'mashable', 'name': 'Mashable'}",Joseph Green,Develop an app from scratch with this impressi...,TL;DR: The Full Stack Web Development course i...,https://mashable.com/uk/shopping/july-18-how-t...,https://mondrian.mashable.com/2020%252F07%252F...,2020-07-18T03:30:00Z,TL;DR: The Full Stack Web Development course i...,1.00000,0.365714,0.400000,0.432727
15,"{'id': 'mashable', 'name': 'Mashable'}",Morgan Sung,The surprising reason Lily Allen's 'Smile' wen...,TikTok users all over the globe are dancing li...,https://mashable.com/article/lily-allen-smile-...,https://mondrian.mashable.com/2020%252F07%252F...,2020-07-18T00:06:14Z,TikTok users all over the globe are dancing li...,0.50000,0.115000,0.143750,0.195455
20,"{'id': 'cnn', 'name': 'CNN'}",Opinion by Michael Bociurkiw,Opinion: How Justin Trudeau's latest ethics sc...,First there was the secretive trip to Bermuda ...,https://www.cnn.com/2020/07/17/opinions/trudea...,https://cdn.cnn.com/cnnnext/dam/assets/1707141...,2020-07-18T02:13:34Z,"Michael Bociurkiw is a global affairs analyst,...",0.50000,0.125000,0.000000,0.093750
23,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Coronavirus: Are mutations making it more infe...,While there have been thousands of changes to ...,https://www.bbc.co.uk/news/health-53325771,https://ichef.bbci.co.uk/news/1024/branded_new...,2020-07-18T23:10:26Z,Image copyrightGetty ImagesImage caption\r\n U...,0.50000,0.000000,-0.238095,-0.035714
27,"{'id': 'reuters', 'name': 'Reuters'}",Reuters Editorial,Get dirty at home: COVID-19 moves South Korea'...,When a pandemic threatens a good romp in the m...,https://www.reuters.com/article/us-health-coro...,https://s2.reutersmedia.net/resources/r/?m=02&...,2020-07-18T12:38:00Z,SEOUL (Reuters) - When a pandemic threatens a ...,-0.60000,0.700000,0.650000,0.350000
28,"{'id': 'reuters', 'name': 'Reuters'}",Reuters Editorial,Get dirty at home: COVID-19 moves S.Korea's mu...,When a pandemic threatens a good romp in the m...,https://www.reuters.com/article/health-coronav...,https://s4.reutersmedia.net/resources_v2/image...,2020-07-18T12:00:00Z,"SEOUL, July 18 (Reuters) - When a pandemic thr...",-0.60000,0.700000,0.650000,0.350000


In [20]:
df2 = df1[((df1['descriptionScore'] >= 0.01) | (df1['description'].isnull()) | (df1['description']=='')) & ((df1['titleScore'] >= 0.01)| (df1['title'].isnull()) | (df1['title']=='')) & ((df1['contentScore'] >= 0.01)| (df1['content'].isnull()) | (df1['content']=='')) & (df1['allScore'] >= 0.2)]
df2


,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
7,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Eric Eldon,Startups Weekly: The TechCrunch List reveals i...,Editor’s note: Get this free weekly recap of T...,http://techcrunch.com/2020/07/18/startups-week...,https://techcrunch.com/wp-content/uploads/2020...,2020-07-18T18:00:22Z,Editors note: Get this free weekly recap of Te...,0.50000,0.359091,0.45,0.405195
11,"{'id': 'mashable', 'name': 'Mashable'}",Joseph Green,Develop an app from scratch with this impressi...,TL;DR: The Full Stack Web Development course i...,https://mashable.com/uk/shopping/july-18-how-t...,https://mondrian.mashable.com/2020%252F07%252F...,2020-07-18T03:30:00Z,TL;DR: The Full Stack Web Development course i...,1.00000,0.365714,0.40,0.432727
31,"{'id': 'reuters', 'name': 'Reuters'}",Reuters Editorial,Tennis: Healthy competition key to Russian suc...,"Russians Daniil Medvedev, Andrey Rublev and Ka...",https://www.reuters.com/article/us-tennis-russ...,https://s2.reutersmedia.net/resources/r/?m=02&...,2020-07-18T09:05:00Z,"(Reuters) - Russians Daniil Medvedev, Andrey R...",0.20000,0.333333,0.50,0.311111
61,"{'id': None, 'name': 'Android Central'}",Jerry Hildenbrand,A little bit of Samsung makes every phone better,Samsung doesn't need to sell its own phones to...,https://www.androidcentral.com/little-bit-sams...,https://www.androidcentral.com/sites/androidce...,2020-07-18T12:00:03Z,Source: Jerry Hildenbrand / Android Central\r\...,0.15625,0.642857,0.50,0.522321
85,"{'id': None, 'name': 'Kotaku'}",Zack Zwiezen,What A Good Boy,This week on Snapshots: Ghost of Tsushima is o...,https://kotaku.com/what-a-good-boy-1844429280,https://i.kinja-img.com/gawker-media/image/upl...,2020-07-18T20:00:00Z,None,0.70000,0.342500,0.00,0.402083


In [21]:
df_must_good = df2.copy()
df_must_good

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
7,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Eric Eldon,Startups Weekly: The TechCrunch List reveals i...,Editor’s note: Get this free weekly recap of T...,http://techcrunch.com/2020/07/18/startups-week...,https://techcrunch.com/wp-content/uploads/2020...,2020-07-18T18:00:22Z,Editors note: Get this free weekly recap of Te...,0.50000,0.359091,0.45,0.405195
11,"{'id': 'mashable', 'name': 'Mashable'}",Joseph Green,Develop an app from scratch with this impressi...,TL;DR: The Full Stack Web Development course i...,https://mashable.com/uk/shopping/july-18-how-t...,https://mondrian.mashable.com/2020%252F07%252F...,2020-07-18T03:30:00Z,TL;DR: The Full Stack Web Development course i...,1.00000,0.365714,0.40,0.432727
31,"{'id': 'reuters', 'name': 'Reuters'}",Reuters Editorial,Tennis: Healthy competition key to Russian suc...,"Russians Daniil Medvedev, Andrey Rublev and Ka...",https://www.reuters.com/article/us-tennis-russ...,https://s2.reutersmedia.net/resources/r/?m=02&...,2020-07-18T09:05:00Z,"(Reuters) - Russians Daniil Medvedev, Andrey R...",0.20000,0.333333,0.50,0.311111
61,"{'id': None, 'name': 'Android Central'}",Jerry Hildenbrand,A little bit of Samsung makes every phone better,Samsung doesn't need to sell its own phones to...,https://www.androidcentral.com/little-bit-sams...,https://www.androidcentral.com/sites/androidce...,2020-07-18T12:00:03Z,Source: Jerry Hildenbrand / Android Central\r\...,0.15625,0.642857,0.50,0.522321
85,"{'id': None, 'name': 'Kotaku'}",Zack Zwiezen,What A Good Boy,This week on Snapshots: Ghost of Tsushima is o...,https://kotaku.com/what-a-good-boy-1844429280,https://i.kinja-img.com/gawker-media/image/upl...,2020-07-18T20:00:00Z,None,0.70000,0.342500,0.00,0.402083


In [22]:
au_entertainment_url = ('http://newsapi.org/v2/top-headlines?'
                        'country=au&'
                        'from=2020-07-18&'
                        'category=entertainment&'
                        'pageSize=100&'
                        'sortBy=popularity&'
                        'apiKey=7cbc46d369ec49efba7d4e1bd3df8b1a')

au_entertainment_response = requests.get(au_entertainment_url)

In [23]:
array_au_entertainment = add_scores_to_array(au_entertainment_response.json()['articles'])
df = pd.DataFrame(array_au_entertainment)
df1 = df[(df['descriptionScore'] >= 0.5) | (df['titleScore'] >= 0.5) | (df['contentScore'] >= 0.5) | (df['allScore'] >= 0.5)]
df1

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
1,"{'id': None, 'name': 'The Sydney Morning Herald'}",None,Royal Family releases Princess Beatrice's wedd...,Princess Beatrice wore a vintage dress loaned ...,https://www.smh.com.au/world/europe/royal-fami...,https://static.ffx.io/images/$zoom_1%2C$multip...,2020-07-18T21:18:00Z,"Beatrice, 31, wore a vintage ivory taffeta dre...",0.000000,0.000000,0.600000,0.600000
3,"{'id': 'abc-news-au', 'name': 'ABC News (AU)'}",None,"Clueless at 25: Like, a totally important teen...",While Clueless' winning combination of writing...,https://www.abc.net.au/news/2020-07-19/clueles...,https://www.abc.net.au/cm/rimage/12464744-16x9...,2020-07-18T19:00:00Z,While many teen films fade away never to be he...,0.200000,0.500000,0.130769,0.203297
8,"{'id': None, 'name': 'Daily Telegraph'}",Karlie Rutherford,Charlize Theron on her body’s most surprising ...,Her peers have awarded her an Oscar. She is on...,https://www.dailytelegraph.com.au/lifestyle/st...,https://cdn.newsapi.com.au/image/v1/a623c50893...,2020-07-18T09:30:00Z,Her peers have awarded her an Oscar. She is on...,0.400000,0.500000,0.616667,0.507143
9,"{'id': None, 'name': 'Daily Mail'}",By J. Peterson For Daily Mail Australia,Rebel Wilson shares flattering selfies of her ...,She has been achieving incredible results thro...,https://www.dailymail.co.uk/tvshowbiz/article-...,https://i.dailymail.co.uk/1s/2020/07/18/09/309...,2020-07-18T09:15:16Z,She has been achieving incredible results thro...,0.000000,0.900000,0.518182,0.484091
14,"{'id': None, 'name': 'PerthNow'}",PerthNow,Ketut takes to social media to confirm he and ...,In what is basically the only good news to com...,https://www.perthnow.com.au/news/offbeat/ketut...,https://images.s.perthnow.com.au/publication/B...,2020-07-18T05:12:00Z,Just as we were about to throw 2020 in a bin f...,0.033333,0.247222,0.600000,0.329167
19,"{'id': None, 'name': 'Yahoo Entertainment'}",Lifestyle Team,COVID hospitalisation for ‘World's most beauti...,Aishwarya Rai Bachchan and her daughter tested...,https://au.lifestyle.yahoo.com/covid-hospitali...,https://s.yimg.com/uu/api/res/1.2/5bCE_7uzSrLb...,2020-07-18T04:12:51Z,Aishwarya Rai Bachchan and her eight-year-old ...,0.675000,0.113636,0.020202,0.233983
26,"{'id': None, 'name': 'Nine'}",Maddison Leach,Jacinda Ardern joins TikTok impersonator for h...,,https://honey.nine.com.au/latest/jacinda-arder...,https://imageresizer.static9.net.au/sff6yYdXUd...,2020-07-18T01:48:00Z,"Jacinda Ardern has teamed up with erm, herself...",0.500000,0.000000,0.445455,0.459091
27,"{'id': None, 'name': 'Mamamia'}",Mamamia Team,The first secret royal wedding in 235 years: W...,"Princess Beatrice wedding venue: On 17 July, P...",https://www.mamamia.com.au/princess-beatrice-w...,https://storage.googleapis.com/mamamia-pwa.app...,2020-07-18T01:09:04Z,"During the knighting ceremony, the Queen share...",-0.075000,0.225000,0.515000,0.221667
37,"{'id': None, 'name': 'Express'}",Josh Saunders,Meghan Markle's stunning assessment of Ivanka ...,MEGHAN MARKLE revealed her dreams of being mor...,https://www.express.co.uk/news/royal/1310970/m...,https://cdn.images.express.co.uk/img/dynamic/1...,2020-07-17T15:59:54Z,None,0.250000,0.500000,0.000000,0.333333
39,"{'id': None, 'name': 'The Sydney Morning Herald'}",Sophie Aubrey,The fashion statement we didn't want might bec...,The mask has become synonymous with the pandem...,https://www.smh.com.au/lifestyle/fashion/the-f...,https://static.ffx.io/images/$zoom_0.5295%2C$m...,2020-07-17T14:00:00Z,Burrows began selling silk masks two months ag...,0.000000,0.037500,0.600000,0.225000


In [24]:
df2 = df1[((df1['descriptionScore'] >= 0.01) | (df1['description'].isnull()) | (df1['description']=='')) & ((df1['titleScore'] >= 0.01)| (df1['title'].isnull()) | (df1['title']=='')) & ((df1['contentScore'] >= 0.01)| (df1['content'].isnull()) | (df1['content']=='')) & (df1['allScore'] >= 0.2)]
df2

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
3,"{'id': 'abc-news-au', 'name': 'ABC News (AU)'}",None,"Clueless at 25: Like, a totally important teen...",While Clueless' winning combination of writing...,https://www.abc.net.au/news/2020-07-19/clueles...,https://www.abc.net.au/cm/rimage/12464744-16x9...,2020-07-18T19:00:00Z,While many teen films fade away never to be he...,0.200000,0.500000,0.130769,0.203297
8,"{'id': None, 'name': 'Daily Telegraph'}",Karlie Rutherford,Charlize Theron on her body’s most surprising ...,Her peers have awarded her an Oscar. She is on...,https://www.dailytelegraph.com.au/lifestyle/st...,https://cdn.newsapi.com.au/image/v1/a623c50893...,2020-07-18T09:30:00Z,Her peers have awarded her an Oscar. She is on...,0.400000,0.500000,0.616667,0.507143
14,"{'id': None, 'name': 'PerthNow'}",PerthNow,Ketut takes to social media to confirm he and ...,In what is basically the only good news to com...,https://www.perthnow.com.au/news/offbeat/ketut...,https://images.s.perthnow.com.au/publication/B...,2020-07-18T05:12:00Z,Just as we were about to throw 2020 in a bin f...,0.033333,0.247222,0.600000,0.329167
19,"{'id': None, 'name': 'Yahoo Entertainment'}",Lifestyle Team,COVID hospitalisation for ‘World's most beauti...,Aishwarya Rai Bachchan and her daughter tested...,https://au.lifestyle.yahoo.com/covid-hospitali...,https://s.yimg.com/uu/api/res/1.2/5bCE_7uzSrLb...,2020-07-18T04:12:51Z,Aishwarya Rai Bachchan and her eight-year-old ...,0.675000,0.113636,0.020202,0.233983
26,"{'id': None, 'name': 'Nine'}",Maddison Leach,Jacinda Ardern joins TikTok impersonator for h...,,https://honey.nine.com.au/latest/jacinda-arder...,https://imageresizer.static9.net.au/sff6yYdXUd...,2020-07-18T01:48:00Z,"Jacinda Ardern has teamed up with erm, herself...",0.500000,0.000000,0.445455,0.459091
37,"{'id': None, 'name': 'Express'}",Josh Saunders,Meghan Markle's stunning assessment of Ivanka ...,MEGHAN MARKLE revealed her dreams of being mor...,https://www.express.co.uk/news/royal/1310970/m...,https://cdn.images.express.co.uk/img/dynamic/1...,2020-07-17T15:59:54Z,None,0.250000,0.500000,0.000000,0.333333
58,"{'id': None, 'name': 'Junkee'}",Joseph Earp,"Delta Goodrem Talks New Record, Tour, And Her ...","""At the core of it, my intentions are to empow...",https://junkee.com/delta-goodrem-interview-par...,https://junkee.com/wp-content/uploads/2020/07/...,2020-07-17T00:46:08Z,"""At the core of it, my intentions are to empow...",0.354978,0.500000,0.265625,0.328429
66,"{'id': None, 'name': 'Daily Mail'}",By Chloe-lee Longhetti For Daily Mail Australia,AAMI hilariously confirms that Rhonda and Ketu...,Everyone remembers the iconic AAMI adverts tha...,https://www.dailymail.co.uk/tvshowbiz/article-...,https://i.dailymail.co.uk/1s/2020/07/16/23/308...,2020-07-16T22:59:13Z,Everyone remembers the iconic AAMI adverts tha...,0.333333,0.500000,0.300000,0.342857
69,"{'id': None, 'name': 'Herald Sun'}",None,Naya Rivera: Incredible tribute to Glee star’s...,Subscribe to the Herald Sun to get unrestricte...,https://www.heraldsun.com.au/subscribe/news/1/,None,2020-07-16T10:53:32Z,None,0.900000,0.250000,0.000000,0.466667


In [25]:
df_au_entainment = df2.copy()
df_au_entainment

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
3,"{'id': 'abc-news-au', 'name': 'ABC News (AU)'}",None,"Clueless at 25: Like, a totally important teen...",While Clueless' winning combination of writing...,https://www.abc.net.au/news/2020-07-19/clueles...,https://www.abc.net.au/cm/rimage/12464744-16x9...,2020-07-18T19:00:00Z,While many teen films fade away never to be he...,0.200000,0.500000,0.130769,0.203297
8,"{'id': None, 'name': 'Daily Telegraph'}",Karlie Rutherford,Charlize Theron on her body’s most surprising ...,Her peers have awarded her an Oscar. She is on...,https://www.dailytelegraph.com.au/lifestyle/st...,https://cdn.newsapi.com.au/image/v1/a623c50893...,2020-07-18T09:30:00Z,Her peers have awarded her an Oscar. She is on...,0.400000,0.500000,0.616667,0.507143
14,"{'id': None, 'name': 'PerthNow'}",PerthNow,Ketut takes to social media to confirm he and ...,In what is basically the only good news to com...,https://www.perthnow.com.au/news/offbeat/ketut...,https://images.s.perthnow.com.au/publication/B...,2020-07-18T05:12:00Z,Just as we were about to throw 2020 in a bin f...,0.033333,0.247222,0.600000,0.329167
19,"{'id': None, 'name': 'Yahoo Entertainment'}",Lifestyle Team,COVID hospitalisation for ‘World's most beauti...,Aishwarya Rai Bachchan and her daughter tested...,https://au.lifestyle.yahoo.com/covid-hospitali...,https://s.yimg.com/uu/api/res/1.2/5bCE_7uzSrLb...,2020-07-18T04:12:51Z,Aishwarya Rai Bachchan and her eight-year-old ...,0.675000,0.113636,0.020202,0.233983
26,"{'id': None, 'name': 'Nine'}",Maddison Leach,Jacinda Ardern joins TikTok impersonator for h...,,https://honey.nine.com.au/latest/jacinda-arder...,https://imageresizer.static9.net.au/sff6yYdXUd...,2020-07-18T01:48:00Z,"Jacinda Ardern has teamed up with erm, herself...",0.500000,0.000000,0.445455,0.459091
37,"{'id': None, 'name': 'Express'}",Josh Saunders,Meghan Markle's stunning assessment of Ivanka ...,MEGHAN MARKLE revealed her dreams of being mor...,https://www.express.co.uk/news/royal/1310970/m...,https://cdn.images.express.co.uk/img/dynamic/1...,2020-07-17T15:59:54Z,None,0.250000,0.500000,0.000000,0.333333
58,"{'id': None, 'name': 'Junkee'}",Joseph Earp,"Delta Goodrem Talks New Record, Tour, And Her ...","""At the core of it, my intentions are to empow...",https://junkee.com/delta-goodrem-interview-par...,https://junkee.com/wp-content/uploads/2020/07/...,2020-07-17T00:46:08Z,"""At the core of it, my intentions are to empow...",0.354978,0.500000,0.265625,0.328429
66,"{'id': None, 'name': 'Daily Mail'}",By Chloe-lee Longhetti For Daily Mail Australia,AAMI hilariously confirms that Rhonda and Ketu...,Everyone remembers the iconic AAMI adverts tha...,https://www.dailymail.co.uk/tvshowbiz/article-...,https://i.dailymail.co.uk/1s/2020/07/16/23/308...,2020-07-16T22:59:13Z,Everyone remembers the iconic AAMI adverts tha...,0.333333,0.500000,0.300000,0.342857
69,"{'id': None, 'name': 'Herald Sun'}",None,Naya Rivera: Incredible tribute to Glee star’s...,Subscribe to the Herald Sun to get unrestricte...,https://www.heraldsun.com.au/subscribe/news/1/,None,2020-07-16T10:53:32Z,None,0.900000,0.250000,0.000000,0.466667


In [27]:
df_total = pd.concat([df_au_entainment, df_good_news, df_must_good, df_top_us, df_total], ignore_index=True, sort=False)
df_total

NameError: name 'df_total' is not defined

In [28]:
df_total = df_au_entainment.append(df_good_news, ignore_index=True, sort=False)
df_total

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
0,"{'id': 'abc-news-au', 'name': 'ABC News (AU)'}",None,"Clueless at 25: Like, a totally important teen...",While Clueless' winning combination of writing...,https://www.abc.net.au/news/2020-07-19/clueles...,https://www.abc.net.au/cm/rimage/12464744-16x9...,2020-07-18T19:00:00Z,While many teen films fade away never to be he...,0.200000,0.500000,0.130769,0.203297
1,"{'id': None, 'name': 'Daily Telegraph'}",Karlie Rutherford,Charlize Theron on her body’s most surprising ...,Her peers have awarded her an Oscar. She is on...,https://www.dailytelegraph.com.au/lifestyle/st...,https://cdn.newsapi.com.au/image/v1/a623c50893...,2020-07-18T09:30:00Z,Her peers have awarded her an Oscar. She is on...,0.400000,0.500000,0.616667,0.507143
2,"{'id': None, 'name': 'PerthNow'}",PerthNow,Ketut takes to social media to confirm he and ...,In what is basically the only good news to com...,https://www.perthnow.com.au/news/offbeat/ketut...,https://images.s.perthnow.com.au/publication/B...,2020-07-18T05:12:00Z,Just as we were about to throw 2020 in a bin f...,0.033333,0.247222,0.600000,0.329167
3,"{'id': None, 'name': 'Yahoo Entertainment'}",Lifestyle Team,COVID hospitalisation for ‘World's most beauti...,Aishwarya Rai Bachchan and her daughter tested...,https://au.lifestyle.yahoo.com/covid-hospitali...,https://s.yimg.com/uu/api/res/1.2/5bCE_7uzSrLb...,2020-07-18T04:12:51Z,Aishwarya Rai Bachchan and her eight-year-old ...,0.675000,0.113636,0.020202,0.233983
4,"{'id': None, 'name': 'Nine'}",Maddison Leach,Jacinda Ardern joins TikTok impersonator for h...,,https://honey.nine.com.au/latest/jacinda-arder...,https://imageresizer.static9.net.au/sff6yYdXUd...,2020-07-18T01:48:00Z,"Jacinda Ardern has teamed up with erm, herself...",0.500000,0.000000,0.445455,0.459091
5,"{'id': None, 'name': 'Express'}",Josh Saunders,Meghan Markle's stunning assessment of Ivanka ...,MEGHAN MARKLE revealed her dreams of being mor...,https://www.express.co.uk/news/royal/1310970/m...,https://cdn.images.express.co.uk/img/dynamic/1...,2020-07-17T15:59:54Z,None,0.250000,0.500000,0.000000,0.333333
6,"{'id': None, 'name': 'Junkee'}",Joseph Earp,"Delta Goodrem Talks New Record, Tour, And Her ...","""At the core of it, my intentions are to empow...",https://junkee.com/delta-goodrem-interview-par...,https://junkee.com/wp-content/uploads/2020/07/...,2020-07-17T00:46:08Z,"""At the core of it, my intentions are to empow...",0.354978,0.500000,0.265625,0.328429
7,"{'id': None, 'name': 'Daily Mail'}",By Chloe-lee Longhetti For Daily Mail Australia,AAMI hilariously confirms that Rhonda and Ketu...,Everyone remembers the iconic AAMI adverts tha...,https://www.dailymail.co.uk/tvshowbiz/article-...,https://i.dailymail.co.uk/1s/2020/07/16/23/308...,2020-07-16T22:59:13Z,Everyone remembers the iconic AAMI adverts tha...,0.333333,0.500000,0.300000,0.342857
8,"{'id': None, 'name': 'Herald Sun'}",None,Naya Rivera: Incredible tribute to Glee star’s...,Subscribe to the Herald Sun to get unrestricte...,https://www.heraldsun.com.au/subscribe/news/1/,None,2020-07-16T10:53:32Z,None,0.900000,0.250000,0.000000,0.466667
9,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Eric Eldon,Startups Weekly: The TechCrunch List reveals i...,Editor’s note: Get this free weekly recap of T...,http://techcrunch.com/2020/07/18/startups-week...,https://techcrunch.com/wp-content/uploads/2020...,2020-07-18T18:00:22Z,Editors note: Get this free weekly recap of Te...,0.500000,0.359091,0.450000,0.405195


In [29]:
df_total_1 = df_total.append(df_must_good, ignore_index=True, sort=False)
df_total_2 = df_total_1.append(df_top_us, ignore_index=True, sort=False)
df_total_3 = df_total_2.append(df_total, ignore_index=True, sort=False)
df_total_3

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
0,"{'id': 'abc-news-au', 'name': 'ABC News (AU)'}",None,"Clueless at 25: Like, a totally important teen...",While Clueless' winning combination of writing...,https://www.abc.net.au/news/2020-07-19/clueles...,https://www.abc.net.au/cm/rimage/12464744-16x9...,2020-07-18T19:00:00Z,While many teen films fade away never to be he...,0.200000,0.500000,0.130769,0.203297
1,"{'id': None, 'name': 'Daily Telegraph'}",Karlie Rutherford,Charlize Theron on her body’s most surprising ...,Her peers have awarded her an Oscar. She is on...,https://www.dailytelegraph.com.au/lifestyle/st...,https://cdn.newsapi.com.au/image/v1/a623c50893...,2020-07-18T09:30:00Z,Her peers have awarded her an Oscar. She is on...,0.400000,0.500000,0.616667,0.507143
2,"{'id': None, 'name': 'PerthNow'}",PerthNow,Ketut takes to social media to confirm he and ...,In what is basically the only good news to com...,https://www.perthnow.com.au/news/offbeat/ketut...,https://images.s.perthnow.com.au/publication/B...,2020-07-18T05:12:00Z,Just as we were about to throw 2020 in a bin f...,0.033333,0.247222,0.600000,0.329167
3,"{'id': None, 'name': 'Yahoo Entertainment'}",Lifestyle Team,COVID hospitalisation for ‘World's most beauti...,Aishwarya Rai Bachchan and her daughter tested...,https://au.lifestyle.yahoo.com/covid-hospitali...,https://s.yimg.com/uu/api/res/1.2/5bCE_7uzSrLb...,2020-07-18T04:12:51Z,Aishwarya Rai Bachchan and her eight-year-old ...,0.675000,0.113636,0.020202,0.233983
4,"{'id': None, 'name': 'Nine'}",Maddison Leach,Jacinda Ardern joins TikTok impersonator for h...,,https://honey.nine.com.au/latest/jacinda-arder...,https://imageresizer.static9.net.au/sff6yYdXUd...,2020-07-18T01:48:00Z,"Jacinda Ardern has teamed up with erm, herself...",0.500000,0.000000,0.445455,0.459091
5,"{'id': None, 'name': 'Express'}",Josh Saunders,Meghan Markle's stunning assessment of Ivanka ...,MEGHAN MARKLE revealed her dreams of being mor...,https://www.express.co.uk/news/royal/1310970/m...,https://cdn.images.express.co.uk/img/dynamic/1...,2020-07-17T15:59:54Z,None,0.250000,0.500000,0.000000,0.333333
6,"{'id': None, 'name': 'Junkee'}",Joseph Earp,"Delta Goodrem Talks New Record, Tour, And Her ...","""At the core of it, my intentions are to empow...",https://junkee.com/delta-goodrem-interview-par...,https://junkee.com/wp-content/uploads/2020/07/...,2020-07-17T00:46:08Z,"""At the core of it, my intentions are to empow...",0.354978,0.500000,0.265625,0.328429
7,"{'id': None, 'name': 'Daily Mail'}",By Chloe-lee Longhetti For Daily Mail Australia,AAMI hilariously confirms that Rhonda and Ketu...,Everyone remembers the iconic AAMI adverts tha...,https://www.dailymail.co.uk/tvshowbiz/article-...,https://i.dailymail.co.uk/1s/2020/07/16/23/308...,2020-07-16T22:59:13Z,Everyone remembers the iconic AAMI adverts tha...,0.333333,0.500000,0.300000,0.342857
8,"{'id': None, 'name': 'Herald Sun'}",None,Naya Rivera: Incredible tribute to Glee star’s...,Subscribe to the Herald Sun to get unrestricte...,https://www.heraldsun.com.au/subscribe/news/1/,None,2020-07-16T10:53:32Z,None,0.900000,0.250000,0.000000,0.466667
9,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Eric Eldon,Startups Weekly: The TechCrunch List reveals i...,Editor’s note: Get this free weekly recap of T...,http://techcrunch.com/2020/07/18/startups-week...,https://techcrunch.com/wp-content/uploads/2020...,2020-07-18T18:00:22Z,Editors note: Get this free weekly recap of Te...,0.500000,0.359091,0.450000,0.405195


In [30]:
modified_df = df_total_3.drop([0,1,3,5,6,7,8,9,12,14,15,16,17,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34])
modified_df

,source,author,title,description,url,urlToImage,publishedAt,content,titleScore,descriptionScore,contentScore,allScore
2,"{'id': None, 'name': 'PerthNow'}",PerthNow,Ketut takes to social media to confirm he and ...,In what is basically the only good news to com...,https://www.perthnow.com.au/news/offbeat/ketut...,https://images.s.perthnow.com.au/publication/B...,2020-07-18T05:12:00Z,Just as we were about to throw 2020 in a bin f...,0.033333,0.247222,0.600000,0.329167
4,"{'id': None, 'name': 'Nine'}",Maddison Leach,Jacinda Ardern joins TikTok impersonator for h...,,https://honey.nine.com.au/latest/jacinda-arder...,https://imageresizer.static9.net.au/sff6yYdXUd...,2020-07-18T01:48:00Z,"Jacinda Ardern has teamed up with erm, herself...",0.500000,0.000000,0.445455,0.459091
10,"{'id': None, 'name': 'Sky Sports'}",Sky Sports,PL Review Podcast: Best Man Utd team since Fer...,Is the current Manchester United team the best...,https://www.skysports.com/football/news/11095/...,https://e1.365dm.com/20/07/1600x900/skysports-...,2020-07-18T15:00:00Z,Is the current Manchester United team the best...,1.000000,0.500000,0.500000,0.571429
11,"{'id': None, 'name': 'Motley Fool'}",newsfeedback@fool.com (Christy Bieber),3 Key Signs More Stimulus Money Is on the Way ...,There's good news for those hoping for more st...,https://www.fool.com/investing/2020/07/18/3-ke...,https://g.foolcdn.com/editorial/images/581129/...,2020-07-18T10:33:00Z,The IRS has sent out more than $269 billion in...,0.066667,0.662500,0.500000,0.360714
13,"{'id': None, 'name': 'The Indian Express'}",Lifestyle Desk,"Happy Sawan Shivratri 2020: Wishes, images, st...","Happy Sawan Shivratri 2020 Wishes Images, Quot...",https://indianexpress.com/article/lifestyle/li...,https://images.indianexpress.com/2020/07/Sawan...,2020-07-18T14:30:33Z,Happy Sawan Shivratri 2020 Wishes Images: Happ...,0.800000,0.650000,0.800000,0.740000
18,"{'id': None, 'name': 'Kotaku'}",Zack Zwiezen,What A Good Boy,This week on Snapshots: Ghost of Tsushima is o...,https://kotaku.com/what-a-good-boy-1844429280,https://i.kinja-img.com/gawker-media/image/upl...,2020-07-18T20:00:00Z,None,0.700000,0.342500,0.000000,0.402083
28,"{'id': None, 'name': 'Daily Mail'}",By Chloe-lee Longhetti For Daily Mail Australia,AAMI hilariously confirms that Rhonda and Ketu...,Everyone remembers the iconic AAMI adverts tha...,https://www.dailymail.co.uk/tvshowbiz/article-...,https://i.dailymail.co.uk/1s/2020/07/16/23/308...,2020-07-16T22:59:13Z,Everyone remembers the iconic AAMI adverts tha...,0.333333,0.500000,0.300000,0.342857


In [31]:
mod_json = modified_df.to_json(orient='records')
mod_json

'[{"source":{"id":null,"name":"PerthNow"},"author":"PerthNow","title":"Ketut takes to social media to confirm he and Rhonda are still together - PerthNow","description":"In what is basically the only good news to come out of 2020 Ketut has taken to social media to confirm he is still with Rhonda!","url":"https:\\/\\/www.perthnow.com.au\\/news\\/offbeat\\/ketut-takes-to-social-media-to-confirm-he-and-rhonda-are-still-together-ng-b881613056z","urlToImage":"https:\\/\\/images.s.perthnow.com.au\\/publication\\/B881613056Z\\/1595049188504_GOS2RELHM.2-1.JPG?imwidth=1024","publishedAt":"2020-07-18T05:12:00Z","content":"Just as we were about to throw 2020 in a bin fire never to be remembered a good news story comes along to prove all is not lost.\\r\\nIn a mystery that has plagued us for more than eight years insurance \\u2026 [+1740 chars]","titleScore":0.0333333333,"descriptionScore":0.2472222222,"contentScore":0.6,"allScore":0.3291666667},{"source":{"id":null,"name":"Nine"},"author":"Maddis